# Colab Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
"""
Change directory to where this file is located
"""
%cd '/content/drive/MyDrive/MLDL1/MLDL1_hw4'

/content/drive/MyDrive/MLDL1/MLDL1_hw4


# Import Modules

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import transforms, datasets

from tqdm.auto import tqdm

# Utils

In [4]:
def train(model, train_loader, optimizer, criterion, DEVICE):
    """
    Trains the model with training data.

    Do NOT modify this function.
    """
    model.train()
    tqdm_bar = tqdm(train_loader)
    for batch_idx, (image, label) in enumerate(tqdm_bar):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        tqdm_bar.set_description("Epoch {} - train loss: {:.6f}".format(epoch, loss.item()))


def evaluate(model, test_loader, criterion, DEVICE):
    """
    Evaluates the trained model with test data.

    Do NOT modify this function.
    """
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in tqdm(test_loader):
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

# ViT Model

In [5]:
###
# Always check tensor shapes!
# Printing shapes can be the fastest way to track the error and fix it.
# Using dropout is optional. You don't have to use pre-declared dropouts.
###

In [6]:
class Patchification(nn.Module):
  """
  (a) Process the batch of images to non-overlapping patches
  Input shape: [batch, channel, height, width]
  Return: [batch, number_of_patches, embedding_dimension]
  """
  def __init__(self, in_channels, patch_size, embedding_dim):
    super().__init__()
    """
    Hint: embedding_dim should be the out_channel of convolution.
    """
    ##### YOUR CODE #####
    self.patch_embedding = nn.Conv2d(in_channels, embedding_dim, kernel_size=patch_size, stride=patch_size)
    #####################

  def forward(self, x):
    ##### YOUR CODE #####
    x = self.patch_embedding(x)
    batch_size, _, height, width = x.size()
    x = x.flatten(2).permute(0, 2, 1)
    #####################
    return x

class Linear_Patchification(nn.Module):
  """
  Extra credit.
  (b) Process the batch of images to non-overlapping patches
  Input shape: [batch, channel, height, width]
  Return: [batch, number_of_patches, embedding_dimension]
  """
  def __init__(self, in_channels, patch_size, embedding_dim):
    super().__init__()
    ##### YOUR CODE #####
    self.patch_size = patch_size
    self.linPatch = nn.Linear(in_channels*patch_size[0]*patch_size[1],embedding_dim)
    #####################

  def forward(self, x):
    ##### YOUR CODE #####
    stored = []
    for i in range(0,x.shape[2]//self.patch_size[0]):
      for j in range(0,x.shape[3]//self.patch_size[1]):
        patch = x[:,:,i*self.patch_size[0]:(i+1)*self.patch_size[0],j*self.patch_size[1]:(j+1)*self.patch_size[1]].reshape(x.shape[0],-1)
        stored.append(patch)
    x = torch.stack(stored,dim=1)
    x = self.linPatch(x)
    #####################
    return x

In [7]:
"""
Hint cell.
You can briefly check if the function is implemented correctly by printing the shape of the output before jumping into the training with GPU.
Example code is written in the below.
"""
batch, channel, height, width = 5, 3, 32, 32
patch_size = (4, 4)
embedding_dim = 128

test_img = torch.ones(batch, channel, height, width)

patchification = Patchification(channel, patch_size, embedding_dim)
after_patchification = patchification(test_img)

print(after_patchification.shape) # Is the shape [batch, number_of_patches, dim] ?

torch.Size([5, 64, 128])


In [8]:
class MLP(nn.Module):
  """
  (c) Feed-forward layer
  Input shape: [batch, number_of_patches, embedding_dimension]
  Return: [batch, number_of_patches, embedding_dimension]
  """
  def __init__(self, dim, hidden_dim, dropout = 0.):
    super().__init__()
    ##### YOUR CODE #####
    self.linear1 = nn.Linear(dim, hidden_dim)
    self.linear2 = nn.Linear(hidden_dim, dim)
    self.dropout = nn.Dropout(dropout)
    self.activation = nn.ReLU()
    #####################

  def forward(self, x):
    ##### YOUR CODE #####
    x = self.activation(self.linear1(x))
    x = self.linear2(x)
    x = self.dropout(x)
    #####################
    return x

In [9]:
class Attention(nn.Module):
  """
  (d) Multi-head attention
  You can implement without considering heads (i.e. vanilla attention).
  However, your maximum score will be 8 points.

  Input shape: [batch, number_of_patches, embedding_dimension]
  Return: [batch, number_of_patches, embedding_dimension]
  """
  def __init__(self, dim, num_heads, dropout = 0.):
    super().__init__()
    """
    Do NOT modify.
    """
    self.head_dim = dim // num_heads
    self.dim = dim
    self.num_heads = num_heads

    self.scale = self.head_dim ** 0.5 # Don't forget scaling!
    # If you are not going to consider head, you need to change self.scale as below.
    # self.scale = self.dim ** 0.5

    self.dropout = nn.Dropout(dropout)

    ##### YOUR CODE #####
    """
    You need to initialize some layers...
    """
    self.query = nn.Linear(dim, dim)
    self.key = nn.Linear(dim, dim)
    self.value = nn.Linear(dim, dim)
    self.out = nn.Linear(dim, dim)
    #####################

  def forward(self, x):
    ##### YOUR CODE #####
    batch_size, num_patches, _ = x.size()
    q = self.query(x)
    k = self.key(x)
    v = self.value(x)

    q = q.view(q.size(0), -1, self.num_heads, self.head_dim).transpose(1, 2)  # [batch, num_heads, num_patches, head_dim]
    k = k.view(k.size(0), -1, self.num_heads, self.head_dim).transpose(1, 2)  # [batch, num_heads, num_patches, head_dim]
    v = v.view(v.size(0), -1, self.num_heads, self.head_dim).transpose(1, 2)  # [batch, num_heads, num_patches, head_dim]

    scores = torch.matmul(q, k.transpose(-2, -1)) / self.scale
    scores = torch.softmax(scores, dim=-1)
    scores = self.dropout(scores)

    attended = torch.matmul(scores, v).transpose(1, 2)  # [batch, num_patches, num_heads, head_dim]
    attended = attended.contiguous().view(attended.size(0), -1, self.dim)  # [batch, num_patches, dim]

    x = self.out(attended)
    #####################
    return x

class Block(nn.Module):
  """
  (e) Attention block
  Input shape: [batch, number_of_patches, embedding_dimension]
  Return: [batch, number_of_patches, embedding_dimension]
  """
  def __init__(self, dim, num_heads, mlp_dim, dropout=0.):
    super().__init__()
    ##### YOUR CODE #####
    self.attention = Attention(dim, num_heads, dropout)
    self.norm1 = nn.LayerNorm(dim)
    self.mlp = MLP(dim, mlp_dim, dropout)
    self.norm2 = nn.LayerNorm(dim)
    #####################

  def forward(self, x):
    """
    Hint: Don't forget the residual connections!
    (Refer the lab3 slide)
    """
    ##### YOUR CODE #####
    residual = x
    x = self.norm1(x)
    x = self.attention(x)
    x = x + residual

    residual = x
    x = self.norm2(x)
    x = self.mlp(x)
    x = x + residual
    #####################
    return x

In [10]:
class ViT(nn.Module):
    def __init__(self, image_shape, patch_size, num_classes, dim, num_heads, depth, mlp_dim, dropout = 0.):
        super().__init__()
        """
        image_shape: [channel, height, width]
        patch_size: [height, width]
        dim: Embedding dimension
        num_heads: Number of heads to be used in Multi-head Attention
        depth: Number of attention blocks to be used
        mlp_dim: Hidden dimension to be used in MLP layer (=feedforward layer)
        """

        image_ch, image_h, image_w = image_shape # image_ch will be 3(RGB 3 channels) for CIFAR10 dataset
        patch_h, patch_w = patch_size

        assert image_h % patch_h == 0 and image_w % patch_w == 0, 'Image height & width must be divisible by those of patch respectively.'
        assert dim % num_heads == 0, 'Embedding dimension should be divisible by number of heads.'

        num_patches = (image_h // patch_h) * (image_w // patch_w) # e.g. [32 x 32] image & [8 x 8] patch size -> [4 x 4 = 16] patches

        # Patchification using convolution.
        self.patchify = Patchification(image_ch, patch_size, dim)

        # Linear Patchification for extra credit.
        # self.patchify = Linear_Patchification(image_ch, patch_size, dim)

        # Learnable positional encoding, 1+ is for class token.
        self.pos_embedding = nn.Parameter(torch.randn(1, 1 + num_patches, dim))

        # Class token which will be prepended to each image.
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))

        # Initialize attention blocks
        self.attention_blocks = nn.ModuleList([
            Block(dim, num_heads, mlp_dim, dropout)
            for _ in range(depth)
        ])

        # Classification head, maps the final vector to class dimension.
        self.classification_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1) # Shape: [batch, 1, dim]
        """
        (f) ViT forward
        Hint!
        After patchification, shape will be [batch, number_of_patches, dim].
        If you successfully prepend cls_tokens to this batch of patchfied images, shape will be [batch, 1+ number_of_patches, dim].
        Then simply add the positional embedding.
        Now the tokens(patches) are ready to go through the attention blocks.
        After attention operation, classify with class token. (Simply take off it from whole tokens)
        """
        ##### YOUR CODE #####
        x = self.patchify(img)
        x = torch.cat((cls_tokens, x), dim = 1)
        x += self.pos_embedding  # Shape: [batch, 1 + num_patches, dim]

        for attention_block in self.attention_blocks:
          x = attention_block(x) # Shape: [batch, 1 + num_patches, dim]

        x = x[:,0]
        x = self.classification_head(x) # Shape: [batch, num_classes]

        #####################
        return x

# ViT Image Classification

In [11]:
"""
Make sure your runtime type is GPU and you are using PyTorch version higher than 1.8!

Do NOT modify.
"""
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using PyTorch version: {}, Device: {}".format(torch.__version__, DEVICE))

Using PyTorch version: 2.0.1+cu118, Device: cuda


In [12]:
"""
You may change some settings including batch size & augmentations.
But if your implementation is correct, default setting is enough to achieve the target performance(i.e. 65%).
"""
BATCH_SIZE = 100

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Prepare Dataset & DataLoader
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
"""
(g) Train your ViT to achieve 65% of accuarcy.
Feel free to change the hyperparameters.
But again, if your implementation is correct, default setting is enough to achieve the target performance(i.e. 65%).
"""
# Hyperparameters
EPOCHS = 10
patch_size = (4,4)
dim = 128
depth = 8
num_heads = 8
mlp_dim = 256
dropout = 0.
learning_rate = 0.001

model = ViT(image_shape = (3,32,32), patch_size = patch_size, num_classes = 10, dim = dim, num_heads = num_heads, depth = depth, mlp_dim = mlp_dim, dropout=dropout).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [14]:
"""
Do NOT modify.
It will take less than 8 minutes for training with default setting.
"""
for epoch in range(1, EPOCHS + 1):
    train(model, trainloader, optimizer, criterion, DEVICE)
    test_loss, test_accuracy = evaluate(model, testloader, criterion, DEVICE)
    print("\n[EPOCH: {}], \tModel: ViT, \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


[EPOCH: 1], 	Model: ViT, 	Test Loss: 0.0135, 	Test Accuracy: 51.16 % 



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]


[EPOCH: 9], 	Model: ViT, 	Test Loss: 0.0094, 	Test Accuracy: 67.03 % 



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


[EPOCH: 10], 	Model: ViT, 	Test Loss: 0.0094, 	Test Accuracy: 66.85 % 

